# Erfocentrum Wegwijzer - Complete RAG Chatbot

Deze notebook bevat ALLES:
1. Website crawlen en scrapen
2. Vector store bouwen
3. Geoptimaliseerde chatbot met geheugen
4. Welkomstflow met privacy-akkoord
5. Doorverwijzing naar Erfolijn

## Stap 1: Packages installeren

In [ ]:
!pip install langchain-core langchain-openai langchain-text-splitters langchain-chroma chromadb openai tiktoken beautifulsoup4 requests lxml -q

## Stap 2: Imports en API key

In [ ]:
import os
import json
import time
import requests
from getpass import getpass
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
from typing import List, Dict
import re

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

## Stap 3: Website crawlen

In [ ]:
def crawl_website(start_url, max_pages=200, delay=0.5):
    """Crawlt de website en verzamelt alle URLs."""
    parsed_start = urlparse(start_url)
    base_domain = parsed_start.netloc
    
    visited = set()
    to_visit = deque([start_url])
    found_urls = []
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    skip_patterns = ['.pdf', '.jpg', '.png', '.gif', '.css', '.js', '/tag/', 'mailto:', 'tel:', 
                    'facebook.com', 'twitter.com', 'linkedin.com', 'instagram.com', 'youtube.com']
    
    print(f"Start crawlen vanaf {start_url}...")
    
    while to_visit and len(found_urls) < max_pages:
        url = to_visit.popleft()
        url = url.split('#')[0].rstrip('/')
        
        if url in visited or any(p in url.lower() for p in skip_patterns):
            continue
        
        visited.add(url)
        
        try:
            response = requests.get(url, headers=headers, timeout=15)
            if 'text/html' not in response.headers.get('Content-Type', ''):
                continue
            response.raise_for_status()
            found_urls.append(url)
            
            if len(found_urls) % 20 == 0:
                print(f"Gevonden: {len(found_urls)} pagina's")
            
            soup = BeautifulSoup(response.content, 'lxml')
            for link in soup.find_all('a', href=True):
                full_url = urljoin(url, link['href'])
                parsed = urlparse(full_url)
                if parsed.netloc == base_domain or parsed.netloc == '':
                    clean_url = f"{parsed.scheme}://{parsed.netloc}{parsed.path}".rstrip('/')
                    if clean_url not in visited:
                        to_visit.append(clean_url)
            
            time.sleep(delay)
        except:
            pass
    
    print(f"Klaar! {len(found_urls)} pagina's gevonden.")
    return found_urls

# Crawl de website
all_urls = crawl_website("https://erfelijkheid.nl/", max_pages=200)

## Stap 4: Content scrapen

In [ ]:
def scrape_page(url):
    """Scraped een pagina en extraheert de tekst."""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept': 'text/html,application/xhtml+xml',
        'Accept-Language': 'nl-NL,nl;q=0.9',
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'lxml')
        
        title_tag = soup.find('title')
        title = title_tag.get_text(strip=True) if title_tag else url
        
        for element in soup(['script', 'style', 'noscript', 'iframe']):
            element.decompose()
        
        body = soup.find('body')
        if body:
            text = body.get_text(separator=' ', strip=True)
        else:
            text = soup.get_text(separator=' ', strip=True)
        
        text = re.sub(r'\s+', ' ', text).strip()
        
        return {
            'url': url,
            'title': title,
            'content': text,
            'success': True
        }
        
    except Exception as e:
        return {'url': url, 'title': '', 'content': '', 'success': False}


def scrape_all_pages(urls, delay=0.5):
    """Scraped alle pagina's."""
    results = []
    total = len(urls)
    
    print(f"Scrapen van {total} pagina's...")
    
    for i, url in enumerate(urls, 1):
        result = scrape_page(url)
        if result['success'] and len(result['content']) > 100:
            results.append(result)
        
        if i % 20 == 0:
            print(f"Voortgang: {i}/{total} - Succesvol: {len(results)}")
        
        time.sleep(delay)
    
    print(f"Klaar! {len(results)} pagina's met content.")
    return results

# Scrape alle pagina's
scraped_pages = scrape_all_pages(all_urls)

In [ ]:
# Optioneel: Opslaan zodat je niet opnieuw hoeft te scrapen
with open('erfocentrum_data.json', 'w', encoding='utf-8') as f:
    json.dump(scraped_pages, f, ensure_ascii=False, indent=2)
print(f"Data opgeslagen: {len(scraped_pages)} pagina's")

## Stap 5: Documenten maken en splitten

In [ ]:
# Maak documenten
documents = []
for page in scraped_pages:
    if page.get('content', '').strip():
        doc = Document(
            page_content=page['content'],
            metadata={'source': page['url'], 'title': page['title']}
        )
        documents.append(doc)

# Split in chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Documenten: {len(documents)}, Chunks: {len(chunks)}")

## Stap 6: Vector store bouwen

In [ ]:
print("Vector store bouwen (dit kost een paar cent)...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
print("Vector store klaar!")

In [ ]:
# LLM instellen
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

## Stap 7: Chatbot met geheugen en doorverwijzing (GEFIXTE VERSIE)

In [ ]:
def chat_met_geheugen(vraag: str, geschiedenis: list) -> tuple:
    """
    Beantwoordt een vraag met geheugen van het gesprek.
    
    BELANGRIJK: De retriever zoekt nu ook op basis van eerdere vragen,
    zodat vervolgvragen zoals 'Is het erfelijk?' goed werken.
    """
    
    # === FIX: Bouw uitgebreide zoekvraag met context uit gesprek ===
    if geschiedenis:
        # Neem de laatste vragen mee voor betere zoekresultaten
        recente_vragen = [v for v, a in geschiedenis[-3:]]
        zoek_query = " ".join(recente_vragen) + " " + vraag
        print(f"   [Debug] Zoekquery: {zoek_query[:80]}...")  # Debug info
    else:
        zoek_query = vraag
    
    # Haal relevante documenten op met de UITGEBREIDE query
    docs = retriever.invoke(zoek_query)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Bouw gespreksgeschiedenis voor de prompt
    geschiedenis_tekst = ""
    if geschiedenis:
        geschiedenis_tekst = "\n\nEERDERE VRAGEN EN ANTWOORDEN IN DIT GESPREK:\n"
        for v, a in geschiedenis[-5:]:
            # Verwijder bronnen uit geschiedenis voor kortere prompt
            a_kort = a.split("\n\n📚")[0].strip()
            geschiedenis_tekst += f"Gebruiker: {v}\nAssistent: {a_kort}\n\n"
    
    # Verzamel bronnen
    bronnen = set()
    for doc in docs:
        bronnen.add(doc.metadata.get('source', ''))
    
    # Prompt met strikte instructies
    prompt = f"""Je bent de Erfocentrum Wegwijzer, een behulpzame assistent die vragen beantwoordt over erfelijkheid en genetica.

BELANGRIJKE REGELS:
1. Beantwoord ALLEEN vragen op basis van de onderstaande context van erfelijkheid.nl
2. Als het antwoord NIET in de context staat, antwoord dan EXACT met alleen: NIET_GEVONDEN
3. Als de vraag NIET over erfelijkheid, genetica of erfelijke aandoeningen gaat, antwoord dan EXACT met alleen: NIET_RELEVANT
4. Geef nooit persoonlijk medisch advies - verwijs naar een (huis)arts voor persoonlijke situaties
5. BELANGRIJK: Kijk naar de gespreksgeschiedenis! Als de gebruiker vraagt "Is het erfelijk?" en de vorige vraag ging over dementie, dan gaat de vraag over dementie.
6. Antwoord altijd in het Nederlands, bondig en duidelijk
{geschiedenis_tekst}
CONTEXT UIT ERFELIJKHEID.NL:
{context}

HUIDIGE VRAAG: {vraag}

Antwoord:"""
    
    # Genereer antwoord
    response = llm.invoke(prompt)
    antwoord = response.content
    
    # Check of doorverwijzing nodig is
    if "NIET_GEVONDEN" in antwoord or "NIET_RELEVANT" in antwoord:
        antwoord = """Helaas kan ik geen antwoord vinden op je vraag in de informatie van erfelijkheid.nl.

Je kunt je vraag stellen aan de deskundigen van het Erfocentrum via de Erfolijn:
👉 https://www.erfelijkheid.nl/contact

Zij helpen je graag verder met persoonlijke vragen over erfelijkheid."""
    else:
        # Voeg bronnen toe
        if bronnen:
            antwoord += "\n\n📚 **Meer informatie:**\n"
            for bron in list(bronnen)[:2]:
                if bron:
                    antwoord += f"- {bron}\n"
    
    # Update geschiedenis
    nieuwe_geschiedenis = geschiedenis + [(vraag, antwoord)]
    
    return antwoord, nieuwe_geschiedenis

## Stap 8: Test de chatbot

In [ ]:
# Test: Gesprek met geheugen - dit zou nu moeten werken!
print("="*60)
print("TEST 1: Gesprek met geheugen (GEFIXTE VERSIE)")
print("="*60)

geschiedenis = []

vraag1 = "Wat is dementie?"
print(f"\n👤 Gebruiker: {vraag1}")
antwoord1, geschiedenis = chat_met_geheugen(vraag1, geschiedenis)
print(f"\n🤖 Wegwijzer: {antwoord1}")

# Vervolgvraag - nu zou hij moeten snappen dat het over dementie gaat!
vraag2 = "Is het erfelijk?"
print(f"\n👤 Gebruiker: {vraag2}")
antwoord2, geschiedenis = chat_met_geheugen(vraag2, geschiedenis)
print(f"\n🤖 Wegwijzer: {antwoord2}")

# Nog een vervolgvraag
vraag3 = "Hoe groot is de kans dat ik het krijg?"
print(f"\n👤 Gebruiker: {vraag3}")
antwoord3, geschiedenis = chat_met_geheugen(vraag3, geschiedenis)
print(f"\n🤖 Wegwijzer: {antwoord3}")

In [ ]:
# Test: Niet-relevante vraag
print("="*60)
print("TEST 2: Niet-relevante vraag")
print("="*60)

vraag_irrelevant = "Wat is de hoofdstad van Frankrijk?"
print(f"\n👤 Gebruiker: {vraag_irrelevant}")
antwoord, _ = chat_met_geheugen(vraag_irrelevant, [])
print(f"\n🤖 Wegwijzer: {antwoord}")

## Stap 9: Volledige interactieve chatbot

In [ ]:
def start_wegwijzer():
    """Start de volledige Erfocentrum Wegwijzer met privacy-flow."""
    
    print("\n" + "="*60)
    print("🧬 ERFOCENTRUM WEGWIJZER")
    print("="*60)
    
    # Stap 1: Welkom
    print("\nStel je vraag aan de Wegwijzer van het Erfocentrum.")
    print("\n[Start chat]")
    input("\nDruk op Enter om te starten...")
    
    # Stap 2: Introductie
    print("\n" + "-"*60)
    print("""Ik help je graag met het zoeken naar algemene informatie 
over erfelijke ziektes of aandoeningen.

⚠️  Let op: voor een persoonlijk medisch advies kan je het 
    beste contact opnemen met je (huis)arts.""")
    
    # Stap 3: Privacy
    print("\n" + "-"*60)
    print("""Wij vinden jouw privacy heel belangrijk. 
Bekijk daarom onze privacyverklaring.

Ga je hiermee akkoord?""")
    
    akkoord = input("\nTyp 'akkoord' of 'niet akkoord': ").lower().strip()
    
    if 'akkoord' not in akkoord or 'niet' in akkoord:
        print("\nJe kunt de chatbot alleen gebruiken als je akkoord gaat.")
        print("Bezoek https://www.erfelijkheid.nl voor meer informatie.")
        return
    
    # Stap 4: Start gesprek
    print("\n" + "-"*60)
    print("Dank voor je akkoord. Waar ben je naar op zoek?")
    print("\n(Typ 'stop' om te stoppen)")
    print("-"*60)
    
    # Chat loop met geheugen
    geschiedenis = []
    
    while True:
        vraag = input("\n👤 Jij: ").strip()
        
        if not vraag:
            continue
        
        if vraag.lower() in ['stop', 'quit', 'exit', 'bye']:
            print("\n🤖 Wegwijzer: Bedankt voor je bezoek! Tot ziens.")
            break
        
        antwoord, geschiedenis = chat_met_geheugen(vraag, geschiedenis)
        print(f"\n🤖 Wegwijzer: {antwoord}")
    
    # Afsluiting
    print("\n" + "="*60)
    print("ℹ️  De antwoorden zijn gebaseerd op informatie van erfelijkheid.nl,")
    print("   zorgvuldig samengesteld en gecontroleerd door medici die")
    print("   aangesloten zijn bij het Erfocentrum.")
    print("\n🔒 Je gesprek is niet opgeslagen - je privacy is gewaarborgd.")
    print("="*60)

# Start!
start_wegwijzer()

---

## ✅ Samenvatting

| Functie | Status |
|---------|--------|
| Titel: Erfocentrum Wegwijzer | ✅ |
| Website scrapen | ✅ |
| Gesprekgeheugen binnen sessie | ✅ GEFIXED |
| Vergeet na sessie (privacy) | ✅ |
| Alleen antwoorden uit website-content | ✅ |
| Welkomstflow met privacy-akkoord | ✅ |
| Doorverwijzing naar Erfolijn | ✅ |
| Disclaimer over medici | ✅ |